# **Modle Behv. _ BASELINE**

# **1.Setup**

In [2]:
import pandas as pd
import numpy as np
import os
from google.colab import drive
from sklearn.metrics import classification_report, f1_score

# 1. 挂载 Google Drive
# 运行后会弹出一个授权窗口，请授权 Colab 访问您的 Drive
drive.mount('/content/drive')

# 2. 定义数据路径
BASE_PATH = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/data/processed'

# 3. 加载数据集 (X, y 分离)
try:
    # 训练集 (已SMOTE和Standard Scale)
    X_train = pd.read_csv(os.path.join(BASE_PATH, 'X_train_smote.csv'))
    y_train = pd.read_csv(os.path.join(BASE_PATH, 'y_train_smote.csv')).squeeze() # .squeeze() 将 DataFrame 转换为 Series

    # 验证集 (已Standard Scale)
    X_val = pd.read_csv(os.path.join(BASE_PATH, 'X_val.csv'))
    y_val = pd.read_csv(os.path.join(BASE_PATH, 'y_val.csv')).squeeze()

    # 测试集（备用，本次基线测试不使用）
    X_test = pd.read_csv(os.path.join(BASE_PATH, 'X_test.csv'))
    y_test = pd.read_csv(os.path.join(BASE_PATH, 'y_test.csv')).squeeze()

    print("✅ 所有数据集加载成功。")
    print(f"训练集样本数 (SMOTE后): {len(X_train)}")
    print(f"验证集样本数: {len(X_val)}")

except FileNotFoundError:
    print(f"🚨 错误：未找到文件。请确保路径 {BASE_PATH} 正确，并且文件名与保存时一致。")
    print("请确认您已经成功运行了上一个回合的保存代码。")
    raise

Mounted at /content/drive
✅ 所有数据集加载成功。
训练集样本数 (SMOTE后): 22858
验证集样本数: 4183


In [3]:
import random
import os
import numpy as np
import torch
import tensorflow as tf  # 如果你用的是TensorFlow/Keras

def seed_everything(seed=42):
    """
    固定所有可能的随机种子，确保实验可复现。
    :param seed: 随机种子数值，通常常用 42, 0, 2023 等
    """
    # 1. Python 原生随机种子
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    # 2. Numpy 随机种子 (数据处理常用)
    np.random.seed(seed)

    # 3. PyTorch 随机种子 (如果你在使用 PyTorch)
    try:
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # 如果使用多GPU

        # 这一步非常重要：强制 CuDNN 使用确定性算法
        # 虽然这会稍微降低训练速度，但能保证结果一致
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    except NameError:
        pass # 如果没装Torch则跳过

    # 4. TensorFlow 随机种子 (如果你在使用 TF/Keras)
    try:
        tf.random.set_seed(seed)
    except NameError:
        pass # 如果没装TF则跳过

    print(f"Global seed set to {seed}")

# --- 使用方法 ---
# 在你开始做任何事情之前（定义模型、切分数据前），先调用它：
seed_everything(42)

Global seed set to 42


# **2.Define Behavioral Features**

In [4]:
# 定义行为特征子集
BEHAVIORAL_FEATURES = [
    'StudyHours',
    'DietHabit_Healthy', 'DietHabit_Moderate', 'DietHabit_Others', 'DietHabit_Unhealthy',
    'SleepDur_5-6 hours', 'SleepDur_7-8 hours', 'SleepDur_Less than 5 hours',
    'SleepDur_More than 8 hours', 'SleepDur_Others'
]

# 从完整数据中提取子集
X_train_behav = X_train[BEHAVIORAL_FEATURES]
X_val_behav = X_val[BEHAVIORAL_FEATURES]

print(f"✅ 行为特征子集已创建，共 {len(X_train_behav.columns)} 个特征。")

✅ 行为特征子集已创建，共 10 个特征。


# **3.Training Model Behv._Baseline**

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, f1_score, roc_auc_score, accuracy_score
import pandas as pd
import numpy as np

# 假设 X_train_behav, y_train, X_val_behav, y_val 变量已在内存中。

# 储存结果
results = {}
val_preds = {}
val_probs = {} # 用于储存概率，以计算 AUC

# --- 定义评估函数 ---
def evaluate_model(model_name, y_true, y_pred, y_prob):
    report = classification_report(y_true, y_pred, output_dict=True)

    # 提取指标
    f1 = report['1']['f1-score']
    recall = report['1']['recall']
    precision = report['1']['precision']
    accuracy = accuracy_report['accuracy'] if 'accuracy' in locals() else accuracy_score(y_true, y_pred)

    # AUC需要概率。对于不能输出概率的模型（如 LSVC），AUC 设为 N/A
    auc = roc_auc_score(y_true, y_prob) if y_prob is not None else 'N/A'

    return {
        'F1-Score (Val)': f1,
        'Recall (Val)': recall,
        'Precision (Val)': precision,
        'Accuracy (Val)': accuracy,
        'AUC (Val)': auc
    }

# ==========================================
# 1. 逻辑回归 (Logistic Regression)
# ==========================================
print("\n--- 1. 训练逻辑回归模型 ---")
lr_model = LogisticRegression(random_state=42, solver='liblinear', class_weight=None)
lr_model.fit(X_train_behav, y_train)
y_val_pred_lr = lr_model.predict(X_val_behav)
y_val_prob_lr = lr_model.predict_proba(X_val_behav)[:, 1] # 获取正类概率

results['LR'] = evaluate_model('LR', y_val, y_val_pred_lr, y_val_prob_lr)
print(f"LR 验证集 F1-Score: {results['LR']['F1-Score (Val)']:.4f}")

# ==========================================
# 2. K-近邻 (K-Nearest Neighbors, k=5)
# ==========================================
print("\n--- 2. 训练 K-近邻模型 (KNN) ---")
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_behav, y_train)
y_val_pred_knn = knn_model.predict(X_val_behav)
y_val_prob_knn = knn_model.predict_proba(X_val_behav)[:, 1] # 获取正类概率

results['KNN'] = evaluate_model('KNN', y_val, y_val_pred_knn, y_val_prob_knn)
print(f"KNN 验证集 F1-Score: {results['KNN']['F1-Score (Val)']:.4f}")

# ==========================================
# 3. 随机森林 (Random Forest)
# ==========================================
print("\n--- 3. 训练随机森林模型 (RF) ---")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=None)
rf_model.fit(X_train_behav, y_train)
y_val_pred_rf = rf_model.predict(X_val_behav)
y_val_prob_rf = rf_model.predict_proba(X_val_behav)[:, 1] # 获取正类概率

results['RF'] = evaluate_model('RF', y_val, y_val_pred_rf, y_val_prob_rf)
print(f"RF 验证集 F1-Score: {results['RF']['F1-Score (Val)']:.4f}")

# ==========================================
# 4. 线性支持向量机 (LinearSVC) - 新增
# ==========================================
print("\n--- 4. 训练线性支持向量机模型 (LSVC) ---")
# LSVC 默认不输出概率，故 AUC 设为 None
lsvc_model = LinearSVC(random_state=42, dual=True, max_iter=5000, class_weight=None)
lsvc_model.fit(X_train_behav, y_train)
y_val_pred_lsvc = lsvc_model.predict(X_val_behav)

results['LSVC'] = evaluate_model('LSVC', y_val, y_val_pred_lsvc, None)
print(f"LSVC 验证集 F1-Score: {results['LSVC']['F1-Score (Val)']:.4f}")

# ==========================================
# 5. 结果总结
# ==========================================
df_results = pd.DataFrame(results).T
df_results = df_results.sort_values(by='F1-Score (Val)', ascending=False)

print("\n==========================================")
print("          基线探索结果总结 (Model Behv.)          ")
print("==========================================")
print("使用特征：仅行为特征 (StudyHours, DietHabit_*, SleepDur_*)")

# 输出为 Markdown 表格，便于记录
print(df_results.round(4).to_markdown())


--- 1. 训练逻辑回归模型 ---
LR 验证集 F1-Score: 0.6744

--- 2. 训练 K-近邻模型 (KNN) ---
KNN 验证集 F1-Score: 0.6553

--- 3. 训练随机森林模型 (RF) ---
RF 验证集 F1-Score: 0.7167

--- 4. 训练线性支持向量机模型 (LSVC) ---
LSVC 验证集 F1-Score: 0.6743

          基线探索结果总结 (Model Behv.)          
使用特征：仅行为特征 (StudyHours, DietHabit_*, SleepDur_*)
|      |   F1-Score (Val) |   Recall (Val) |   Precision (Val) |   Accuracy (Val) | AUC (Val)          |
|:-----|-----------------:|---------------:|------------------:|-----------------:|:-------------------|
| RF   |         0.716651 |       0.768571 |          0.671301 |         0.644035 | 0.6781549042005722 |
| LR   |         0.674414 |       0.651429 |          0.69908  |         0.631604 | 0.6789012800734835 |
| LSVC |         0.674271 |       0.651429 |          0.698774 |         0.631365 | N/A                |
| KNN  |         0.655261 |       0.68     |          0.632258 |         0.580923 | 0.5809357372493139 |


In [7]:
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, f1_score, roc_auc_score, accuracy_score
import numpy as np
import os
import sys

# ==========================================
# 0. 重新安装 CatBoost 库 (保留安装代码)
# ==========================================
print("--- 正在安装 CatBoost 库 ---")
try:
    # 尝试安装
    # ⚠️ 实际运行中如果环境不支持，请注释掉 pip install
    # !pip install catboost --quiet
    # 检查是否已安装
    if 'catboost' not in sys.modules:
        print("🚨 CatBoost 未安装。请手动安装后再运行。")
        # return # 在实际环境中应返回或抛出错误
except Exception as e:
    print(f"🚨 CatBoost 安装或检查失败: {e}")
    # return

# 假设 X_train, y_train, X_val, y_val 变量已在内存中且已缩放。

# 1. 准备行为特征子集
BEHAVIORAL_FEATURES = [
    'StudyHours', 'DietHabit_Healthy', 'DietHabit_Moderate', 'DietHabit_Others', 'DietHabit_Unhealthy',
    'SleepDur_5-6 hours', 'SleepDur_7-8 hours', 'SleepDur_Less than 5 hours',
    'SleepDur_More than 8 hours', 'SleepDur_Others'
]
X_train_behav = X_train[BEHAVIORAL_FEATURES]
X_val_behav = X_val[BEHAVIORAL_FEATURES]

# 储存结果
results = {} # 用于存储所有模型的指标

# --- 定义通用的评估函数 ---
def evaluate_and_store(model_name, model, X_val, y_val):
    y_val_pred = model.predict(X_val)

    # 尝试获取概率，用于 AUC
    y_val_prob = None
    try:
        if isinstance(model, LinearSVC): # LSVC 需要 special handling
            y_val_prob = None
        else:
            y_val_prob = model.predict_proba(X_val)[:, 1]
    except AttributeError:
        y_val_prob = None

    report = classification_report(y_val, y_val_pred, output_dict=True)

    # 提取指标
    f1 = report['1']['f1-score']
    recall = report['1']['recall']
    precision = report['1']['precision']
    accuracy = accuracy_score(y_val, y_val_pred)
    auc = roc_auc_score(y_val, y_val_prob) if y_val_prob is not None else 'N/A'

    results[model_name] = {
        'F1-Score (Val)': f1,
        'Recall (Val)': recall,
        'Precision (Val)': precision,
        'Accuracy (Val)': accuracy,
        'AUC (Val)': auc
    }
    print(f"{model_name} 验证集 F1-Score: {f1:.4f}")

# ==========================================
# A. 训练简单基线模型 (LR, KNN, RF, LSVC)
# ==========================================
print("\n--- A. 训练简单基线模型 ---")

# 1. 逻辑回归 (LR)
lr_model = LogisticRegression(random_state=42, solver='liblinear', class_weight=None)
lr_model.fit(X_train_behav, y_train)
evaluate_and_store('LR_Behav', lr_model, X_val_behav, y_val)

# 2. K-近邻 (KNN)
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_behav, y_train)
evaluate_and_store('KNN_Behav', knn_model, X_val_behav, y_val)

# 3. 随机森林 (RF)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=None)
rf_model.fit(X_train_behav, y_train)
evaluate_and_store('RF_Behav', rf_model, X_val_behav, y_val)

# 4. 线性支持向量机 (LSVC)
lsvc_model = LinearSVC(random_state=42, dual=True, max_iter=5000, class_weight=None)
lsvc_model.fit(X_train_behav, y_train)
evaluate_and_store('LSVC_Behav', lsvc_model, X_val_behav, y_val)

# ==========================================
# B. 训练高级树模型 (XGB, LGBM, CB)
# ==========================================
print("\n--- B. 训练高级树模型 ---")

# 5. XGBoost (XGB)
xgb_behav_model = xgb.XGBClassifier(
    objective='binary:logistic', eval_metric='logloss', use_label_encoder=False,
    n_estimators=100, random_state=42, n_jobs=-1, verbose=0 # 增加 verbose=0 保持输出简洁
)
xgb_behav_model.fit(X_train_behav, y_train)
evaluate_and_store('XGB_Behav', xgb_behav_model, X_val_behav, y_val)

# 6. LightGBM (LGBM)
lgbm_behav_model = lgb.LGBMClassifier(
    objective='binary', metric='binary_logloss', n_estimators=100,
    random_state=42, n_jobs=-1, verbose=-1
)
lgbm_behav_model.fit(X_train_behav, y_train)
evaluate_and_store('LGBM_Behav', lgbm_behav_model, X_val_behav, y_val)

# 7. CatBoost (CB)
cb_behav_model = CatBoostClassifier(
    iterations=100, loss_function='Logloss', eval_metric='F1',
    random_seed=42, verbose=0, thread_count=-1
)
cb_behav_model.fit(X_train_behav, y_train)
evaluate_and_store('CB_Behav', cb_behav_model, X_val_behav, y_val)


# ==========================================
# C. 结果总结与 Insight 分析
# ==========================================

# 整合所有基线结果
df_comparison_behav = pd.DataFrame(results).T
df_comparison_behav = df_comparison_behav.sort_values(by='F1-Score (Val)', ascending=False)

print("\n==========================================")
print("     🚀 行为特征模型最终比较 (Model Behv. Final)     ")
print("==========================================")
print("对比目标：确定学生模型训练的性能目标（F1 > 0.7167）。")

# 输出为 Markdown 表格
print(df_comparison_behav.round(4).to_markdown())

# 确定新的最强基线
strongest_behav_baseline_name = df_comparison_behav.index[0]
strongest_behav_f1 = df_comparison_behav.iloc[0]['F1-Score (Val)']

print(f"\n🎉 行为特征的最高性能基线为: {strongest_behav_baseline_name}")
print(f"新的最强基线 F1-Score: {strongest_behav_f1:.4f}")
print("--- 知识蒸馏 (KD) 目标设定: 学生模型 F1 必须超越 0.7167 ---")

--- 正在安装 CatBoost 库 ---

--- A. 训练简单基线模型 ---
LR_Behav 验证集 F1-Score: 0.6744
KNN_Behav 验证集 F1-Score: 0.6553
RF_Behav 验证集 F1-Score: 0.7167
LSVC_Behav 验证集 F1-Score: 0.6743

--- B. 训练高级树模型 ---


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:199: UserWarning: [06:14:32] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder", "verbose" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGB_Behav 验证集 F1-Score: 0.7079
LGBM_Behav 验证集 F1-Score: 0.7157
CB_Behav 验证集 F1-Score: 0.7151

     🚀 行为特征模型最终比较 (Model Behv. Final)     
对比目标：确定学生模型训练的性能目标（F1 > 0.7167）。
|            |   F1-Score (Val) |   Recall (Val) |   Precision (Val) |   Accuracy (Val) | AUC (Val)          |
|:-----------|-----------------:|---------------:|------------------:|-----------------:|:-------------------|
| RF_Behav   |         0.716651 |       0.768571 |          0.671301 |         0.644035 | 0.6781549042005722 |
| LGBM_Behav |         0.715709 |       0.762449 |          0.674368 |         0.645231 | 0.6791064215645866 |
| CB_Behav   |         0.715107 |       0.761224 |          0.674259 |         0.644753 | 0.678735000058881  |
| XGB_Behav  |         0.70792  |       0.742449 |          0.67646  |         0.641167 | 0.6776753771329651 |
| LR_Behav   |         0.674414 |       0.651429 |          0.69908  |         0.631604 | 0.6789012800734835 |
| LSVC_Behav |         0.674271 |       0.651429 |   

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import classification_report, f1_score, accuracy_score, roc_auc_score, precision_score, recall_score
import numpy as np
import tensorflow as tf
import pandas as pd

# ---------------------------
# 数据准备 (假设 X_train_behav, y_train, X_val_behav, y_val 已定义)
# ---------------------------
X_train_nn = X_train_behav
X_val_nn = X_val_behav
CHECKPOINT_PATH_16_8 = 'best_model_16_8_student.weights.h5'

# 转换标签为 Keras 兼容的 2D 格式 (虽然这里是 1D 输出，但保持一致性)
y_train_2d = np.expand_dims(np.array(y_train), axis=1)
y_val_2d = np.expand_dims(np.array(y_val), axis=1)
y_val_1d = np.array(y_val) # 用于 sklearn 评估

# ---------------------------
# 回调函数
# ---------------------------
# 使用 F1 作为监控指标
early_stopping = EarlyStopping(monitor='val_f1_score', patience=15, mode='max', restore_best_weights=True)
model_checkpoint = ModelCheckpoint(CHECKPOINT_PATH_16_8, monitor='val_f1_score', mode='max', save_best_only=True, save_weights_only=True, verbose=0)

# ---------------------------
# 学生 NN 结构: 16 -> 8
# ---------------------------
nn_16_8_model = Sequential([
    Dense(16, activation='tanh', input_shape=(X_train_nn.shape[1],)),
    Dropout(0.1),
    Dense(8, activation='tanh', input_shape=(X_train_nn.shape[1],)),
    Dropout(0.1),
    Dense(1, activation='sigmoid')
])

# 学习率调整到 0.0005 (与最高 AUC 教师模型配置一致)
nn_16_8_model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.F1Score(name='f1_score', average='micro', threshold=0.5)]
)

# ---------------------------
# 训练模型
# ---------------------------
print("\n--- 正在训练 学生 NN (结构: 16 -> 8) ---")
history_16_8 = nn_16_8_model.fit(
    X_train_nn,
    y_train_2d,
    epochs=200,
    batch_size=32,
    validation_data=(X_val_nn, y_val_2d),
    callbacks=[early_stopping, model_checkpoint],
    verbose=1
)

nn_16_8_model.load_weights(CHECKPOINT_PATH_16_8)

# ---------------------------
# 评估并生成完整报告
# ---------------------------
y_val_prob_16_8 = nn_16_8_model.predict(X_val_nn, verbose=0).flatten()
y_val_pred_16_8 = (y_val_prob_16_8 > 0.5).astype("int32")

# 计算所有关键指标
f1_16_8 = f1_score(y_val_1d, y_val_pred_16_8)
prec_16_8 = precision_score(y_val_1d, y_val_pred_16_8)
recall_16_8 = recall_score(y_val_1d, y_val_pred_16_8)
acc_16_8 = accuracy_score(y_val_1d, y_val_pred_16_8)
auc_16_8 = roc_auc_score(y_val_1d, y_val_prob_16_8)

# 创建结果 DataFrame
df_nn_student_result = pd.DataFrame({
    'Model': ['NN_Student (Hard Labels)'],
    'F1-Score (Val)': [f1_16_8],
    'Recall (Val)': [recall_16_8],
    'Precision (Val)': [prec_16_8],
    'Accuracy (Val)': [acc_16_8],
    'AUC (Val)': [auc_16_8]
}).set_index('Model')

print("\n==============================================")
print("     NN 学生模型 (16-8) 仅硬标签训练性能    ")
print("==============================================")

# 输出完整指标的 Markdown 表格
print(df_nn_student_result.round(4).to_markdown())

# 设定基线目标
RF_BASELINE_F1 = 0.7167

print(f"\n🚀 **性能目标对比 (RF 基线 F1): {RF_BASELINE_F1}**")

if f1_16_8 > RF_BASELINE_F1:
    print(f"🎉 恭喜！模型 F1 ({f1_16_8:.4f}) 已超越 RF 基线！")
else:
    print(f"⚠️ 模型 F1 ({f1_16_8:.4f}) 仍低于 RF 基线。**知识蒸馏至关重要！**")


--- 正在训练 学生 NN (结构: 16 -> 8) ---
Epoch 1/200


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


715/715 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.6190 - f1_score: 0.6211 - loss: 0.6515 - val_accuracy: 0.6285 - val_f1_score: 0.6760 - val_loss: 0.6397
Epoch 2/200
715/715 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6254 - f1_score: 0.6206 - loss: 0.6433 - val_accuracy: 0.6419 - val_f1_score: 0.7006 - val_loss: 0.6350
Epoch 3/200
715/715 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6281 - f1_score: 0.6243 - loss: 0.6413 - val_accuracy: 0.6383 - val_f1_score: 0.7126 - val_loss: 0.6349
Epoch 4/200
715/715 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6310 - f1_score: 0.6308 - loss: 0.6392 - val_accuracy: 0.6400 - val_f1_score: 0.6987 - val_loss: 0.6342
Epoch 5/200
715/715 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6331 - f1_score: 0.6324 - loss: 0.6391 - val_accuracy: 0.6424 - val_f1_score: 0.7124 - val_loss: 0.6334
Epoch 6/200
715/715 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6331 - f1_score: 0.6305 - loss: 0.6365 - val_accuracy: 0.6414 - val_f1_score: 0.7087 - val

In [34]:
import os
import tensorflow as tf
import numpy as np

# ------------------------------------------------
# 1. 定义路径和模型名称
# ------------------------------------------------
# Google Drive 的目标存储路径
BASE_DRIVE_PATH = '/content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/models'
# 最佳硬标签学生模型 (F1: 0.7281) 的保存路径
HARD_STUDENT_SAVE_PATH = os.path.join(BASE_DRIVE_PATH, 'nn_student_best_hard_baseline.keras')

# --- 2. 验证目录是否存在 ---
if not os.path.exists(BASE_DRIVE_PATH):
    print(f"⚠️ 目录 {BASE_DRIVE_PATH} 不存在，正在创建...")
    os.makedirs(BASE_DRIVE_PATH, exist_ok=True)
else:
    print(f"✅ 目标目录 {BASE_DRIVE_PATH} 已存在。")

# ------------------------------------------------
# 3. 💾 保存最佳硬标签学生模型
# ------------------------------------------------
try:
    # 假设您的最佳模型对象是 nn_16_8_model
    # 并且它已经加载了 F1=0.7281 的最佳权重

    # Keras 模型的 save() 方法会保存结构、权重和编译配置
    nn_16_8_model.save(HARD_STUDENT_SAVE_PATH)

    print(f"\n✅ 最佳硬标签学生模型已成功保存到:")
    print(f"   路径: {HARD_STUDENT_SAVE_PATH}")
    print("\n请在新 Notebook 中通过 tf.keras.models.load_model() 加载此模型。")

except NameError:
    print("\n❌ 错误: nn_16_8_model 变量未定义。")
    print("请确保在运行此代码块之前，您已经训练并加载了最佳权重的模型对象。")
except Exception as e:
    print(f"\n❌ 错误: 模型保存失败。请检查 Google Drive 挂载和文件权限。错误信息: {e}")

✅ 目标目录 /content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/models 已存在。

✅ 最佳硬标签学生模型已成功保存到:
   路径: /content/drive/MyDrive/CA6001_DEPRESSION_PROJECT/models/nn_student_best_hard_baseline.keras

请在新 Notebook 中通过 tf.keras.models.load_model() 加载此模型。
